### 파이썬 크롤링
> 데이터를 자동으로 저장하는 실습을 위해, 크롤링 기술을 활용<br>
> 파이썬 입문과 크롤링 학습에서 익힌 크롤링 기술을 활용

In [ ]:
import requests
from bs4 import BeautifulSoup

for page_num in range(10):
    if page_num == 0:
        res = requests.get('https://davelee-fun.github.io/')
    else:
        res = requests.get('https://davelee-fun.github.io/page' + str(page_num + 1))
    soup = BeautifulSoup(res.content, 'html.parser')
    data = soup.select('div.card-body')
    for item in data:
        category = item.select_one('h2.card-title').get_text().replace('관련 상품 추천', '').strip()
        product = item.select_one('h4.card-text').get_text().replace('상품명:', '').strip()
        print (category, product)

### Schema 구성 (필요 없음)

### 크롤링 + MongoDB 저장

In [ ]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient


client = MongoClient("mongodb://localhost:27017")
db = client['test']
collection = db.daveshop

for page_num in range(10):
    if page_num == 0:
        res = requests.get('https://davelee-fun.github.io/')
    else:
        res = requests.get('https://davelee-fun.github.io/page' + str(page_num + 1))
    soup = BeautifulSoup(res.content, 'html.parser')
    data = soup.select('div.card-body')
    for item in data:
        category = item.select_one('h2.card-title').get_text().replace('관련 상품 추천', '').strip()
        product = item.select_one('h4.card-text').get_text().replace('상품명:', '').strip()
        data = dict()
        data['category'] = category
        data['product'] = product
        collection.insert_one(data)
        
client.close() 

### MongoDB 데이터 읽기

In [ ]:
from pymongo import MongoClient

# MongoDB에 연결 (인증 미필요시)
client = MongoClient("mongodb://localhost:27017")

db = client['test']
collection = db.daveshop

documents = collection.find()  
for document in documents:
    print(document)

client.close()

### 연습문제를 위한 사전 준비

In [5]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")

db = client['test']
collection = db.daveshop

<div class="alert alert-block" style="border: 2px solid #1565C0;background-color:#E3F2FD;padding:10px">
<font size="3em" style="color:#0D47A1;">연습문제: 'category' 필드에 있는 모든 카테고리 종류를 조회하세요 (중복제거)</font><br>
</div>

In [6]:
category = collection.distinct("category")
print(category)

['매트리스커버', '여름이불세트', '행거도어']


<div class="alert alert-block" style="border: 2px solid #1565C0;background-color:#E3F2FD;padding:10px">
<font size="3em" style="color:#0D47A1;">연습문제: 모든 상품(product)을 조회하세요</font><br>
</div>

In [ ]:
products = collection.find({},{"_id":0, "product":1})
for product in products: print(product)

<div class="alert alert-block" style="border: 2px solid #1565C0;background-color:#E3F2FD;padding:10px">
<font size="3em" style="color:#0D47A1;">연습문제: "매트리스커버" 카테고리의 모든 상품을 조회하세요<br> (find 와 aggregation 을 사용한 코드를 각각 작성해주세요)</font><br>
</div>

In [ ]:
for item in collection.find({"category":{"$eq": "매트리스커버"}},{"_id":0, "product":1}): print(item)

In [ ]:
pipeline = [
    {"$match": {"category": { "$eq": "매트리스커버"}}},
    {"$project": {"_id":0, "product":1}}
]
for item in collection.aggregate(pipeline): print(item)

<div class="alert alert-block" style="border: 2px solid #1565C0;background-color:#E3F2FD;padding:10px">
<font size="3em" style="color:#0D47A1;">연습문제: 각 카테고리별 상품 수를 조회하세요. 이를 위해 MongoDB의 aggregation 기능을 사용하세요</font><br>
</div>

In [38]:
pipeline=[
    {"$group": {"_id": "$category", "count": {"$sum":1}}},
    {"$project": {"_id":0, "카테고리":"$_id", "상품 수":"$count"}}
]
for item in collection.aggregate(pipeline): print(item)

{'카테고리': '여름이불세트', '상품 수': 31}
{'카테고리': '매트리스커버', '상품 수': 31}
{'카테고리': '행거도어', '상품 수': 32}


<div class="alert alert-block" style="border: 2px solid #1565C0;background-color:#E3F2FD;padding:10px">
<font size="3em" style="color:#0D47A1;">연습문제: 상품명(product)에 '핑크'가 포함된 모든 상품을 조회하세요</font><br>
</div>

In [39]:
import re

regux = re.compile("핑크")

for item in collection.find({"product":regux}, {"_id":0, "상품명: ":"$product"}):
    print(item)

{'상품명: ': '보드래 헬로우 누빔 매트리스커버, 핑크'}
{'상품명: ': '보드래 퍼펙트 누빔 매트리스커버, 베이지핑크'}
{'상품명: ': '쉬즈홈 모던그리드 순면 여름이불 베개커버 패드세트, 핑크'}
{'상품명: ': '보드래 헬로우 누빔 매트리스커버, 핑크'}
{'상품명: ': '더자리 에코항균 마이크로 매트리스커버, 밀키핑크'}


<div class="alert alert-block" style="border: 2px solid #1565C0;background-color:#E3F2FD;padding:10px">
<font size="3em" style="color:#0D47A1;">연습문제: '매트리스커버' 카테고리의 상품 중, 상품명에 '차콜'이 포함된 상품을 조회하세요</font><br>
</div>

In [43]:
import re

getColor = re.compile("차콜")

for item in collection.find({"$and": [{"category": {"$eq": "매트리스커버"}}, {"product": getColor}]},
                           {"_id":0, "상품명": "$product"}):
    print(item)

{'상품명': '보드래 퍼펙트 누빔 매트리스커버, 차콜'}
{'상품명': '더자리 에코항균 마이크로 매트리스커버, 밀키차콜그레이'}
{'상품명': '더자리 프레쉬 퓨어 매트리스 커버, 퓨어 차콜그레이'}
{'상품명': '바숨 순면 누빔 침대 매트리스커버, 차콜'}
{'상품명': '쿠팡 브랜드 - 코멧 홈 순면 매트리스커버, 차콜그레이'}


<div class="alert alert-block" style="border: 2px solid #1565C0;background-color:#E3F2FD;padding:10px">
<font size="3em" style="color:#0D47A1;">연습문제: 'product' 필드에 대한 텍스트 인덱스를 생성하세요</font><br>
</div>

In [45]:
result = collection.create_index([("product","text")])
print(result)

product_text


<div class="alert alert-block" style="border: 2px solid #1565C0;background-color:#E3F2FD;padding:10px">
<font size="3em" style="color:#0D47A1;">연습문제: 'product' 필드에서 '누빔'이라는 단어를 포함하는 모든 문서를 텍스트 검색을 이용해 찾으세요<br> (find 와 aggregation 을 사용한 코드를 각각 작성해주세요)</font><br>
</div>

In [47]:
for item in collection.find({"$text": {"$search": "누빔"}}, {"_id":0, "상품명": "$product"}):
    print(item)

{'상품명': '보드래 헬로우 누빔 매트리스커버, 핑크'}
{'상품명': '보드래 퍼펙트 누빔 매트리스커버, 베이지핑크'}
{'상품명': '보드래 퍼펙트 누빔 매트리스커버, 차콜'}
{'상품명': '보드래 헬로우 누빔 매트리스커버, 핑크'}
{'상품명': '바숨 순면 누빔 침대 매트리스커버, 화이트'}
{'상품명': '바숨 순면 누빔 침대 매트리스커버, 차콜'}
{'상품명': '보드래 국내산 순면 60수 누빔 매트리스커버, 그레이'}
{'상품명': '타카타카 프리미엄 나노 화이바 누빔 매트리스 커버, 미스밍고'}
{'상품명': '스코홈 어번시리즈 순면 차렵이불 누빔 매트커버세트 S 차콜'}
{'상품명': '타카타카 프리미엄 나노 화이바 누빔 매트리스 커버, 프렌치불독'}
{'상품명': '타카타카 프리미엄 나노 화이바 누빔 매트리스 커버, 젠틀핑핑'}
{'상품명': '쿠팡 브랜드 - 코멧 홈 40수 순면 누빔 매트리스커버, 챠콜'}


In [53]:
pipeline = [
    {"$match": {"product": {"$in": "누빔" } } },
    {"$project": {"_id":0, "상품명":"$product"}}
]
for item in collection.aggregate(pipeline): print(item)

OperationFailure: unknown top level operator: $product. If you have a field name that starts with a '$' symbol, consider using $getField or $setField., full error: {'ok': 0.0, 'errmsg': "unknown top level operator: $product. If you have a field name that starts with a '$' symbol, consider using $getField or $setField.", 'code': 2, 'codeName': 'BadValue'}